In [1]:
import numpy as np
import torch 
import sys
sys.path.append("../")
import scipy

import pandas as pd
import matplotlib.pyplot as plt
# import utils_BayTIDE as utils 
from scipy.io import loadmat

In [2]:
# impoet tensor.mat using scipy
tensor = loadmat('../raw/tensor.mat')
tensor = tensor['tensor']
print(tensor.shape)

# only use the first 5 days, resahpe to 2D
# sub_tensor = tensor[:,:5,:] 
sub_tensor = tensor.reshape(214,61*144)
print(sub_tensor.shape)

# to match the TIDER, just use the first 500 time points
sub_tensor = sub_tensor[:,-501:-1]
print(sub_tensor.shape)

(214, 61, 144)
(214, 8784)
(214, 500)


In [7]:
# check the zero ratio
print(np.sum(sub_tensor==0)/sub_tensor.size)

0.0


In [17]:
L_test = 10
mean_pred = np.ones(sub_tensor.shape)*np.mean(sub_tensor[:,:-L_test])

mask = np.ones(sub_tensor.shape)
mask[:,:-L_test] = 0

mean_pred = mean_pred*mask
tensor_mask = sub_tensor*mask

# compute the RMSE and MAE for the mean prediction
print(np.sqrt(np.sum((tensor_mask-mean_pred)**2)/np.sum(mask)))
print(np.sum(np.abs(tensor_mask-mean_pred))/np.sum(mask))

9.656293863902315
7.522097348525228


In [14]:
data_save = {}
data_save['ndims'] = sub_tensor.shape

data_save['raw_data'] = sub_tensor

data_save['data'] = []

# sorted unique timestamps, set as (normalized to 0-1 ) regular time intervals at current stage
data_save['time_uni'] = np.linspace(0,1,sub_tensor.shape[1])

def generate_forecast_mask( shape, test_ratio= 0.1):
    """
    test_ratio: the ratio of test data (last L step to forecast)
    """
    N,T = shape

    mask_train_list = []
    mask_test_list = []
    mask_valid_list = []

    mask = np.ones((N,T))

    L_train = int(T*(1-test_ratio))
    L_test = T-L_train

    mask_train = np.ones((N,T))
    mask_train[:,L_train:] = 0

    mask_test = np.ones((N,T))
    mask_test[:,:L_train] = 0

    mask_valid = np.zeros((N,T))

    return mask_train, mask_test, mask_valid

fold = 1
test_rate = 0.006


for i in range(fold):
    mask_train, mask_test, mask_valid = generate_forecast_mask(sub_tensor.shape, test_rate)
    data_save['data'].append({'mask_train':mask_train, 'mask_test':mask_test, 'mask_valid':mask_valid})

file_name = '../guangzhou/guangzhou_forecast'+'_r_%.3f'%(test_rate)+'.npy'
np.save(file_name, data_save)


In [9]:
mask_test.sum()/mask_train.size

0.1

In [10]:
import numpy as np
import torch

np.arange(1,10)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])